In [1]:
import pandas as pd
import numpy as np
import random

import openai
import os
from tqdm import tqdm
import time

import sys

sys.path.insert(0, '..')

In [2]:
# TURBO COST
TURBO_COST = 0.00200 / 1000

In [3]:
# Initialize the API client
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [4]:
df = pd.read_csv('../data/seed/youtube/seed_videos.csv')

In [5]:
def create_messages(snippet):
    messages = [
        {"role": "system", "content" : "You are a classifier that determines if a YouTube video snippet falls under a label. A snippet is a concatenation of the video title, summarized transcript, and video tags. The labels and additional instructions will be included in the first user message."},
        {"role": "user", "content" : """Labels:

- Blockchain: Cryptocurrency, NFTs, or other Web3 investments
- Traditional: Stocks, Bonds, Real Estate, Commodities, or other traditional investments
- Mixed: Videos that recommend or educate about both blockchain and traditional investments
- None: Videos that do not recommend or educate about either blockchain or traditional investments

Instructions:

- If a video snippet mentions keywords related to blockchain or traditional investments but does not recommend or educate about the label, it will be classified as None.
- Please return predictions in the format" {Label} : {50 word or shorter rationale}"""},
        {"role": "assistant", "content": """Understood. I will classify YouTube video snippets based on the provided labels and instructions. Here's how I will format the predictions:

{Label} : {50-word or shorter rationale}

Please provide me with the YouTube video snippet you would like me to classify."""},
    ]
    
    snippet_message = {"role" : "user", "content" : snippet}
    
    messages.append(snippet_message)
    
    return messages

In [6]:
def classify(messages):
    chatCompletion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=0.25
    )
    return chatCompletion

In [7]:
messages = create_messages(df['snippet'][0])

In [8]:
chatCompletion = classify(messages)

In [9]:
chatCompletion

<OpenAIObject chat.completion id=chatcmpl-6ppMdZMjb0Pjkf2ooBsYVd6ZSBPno at 0x7f9f21f0eb30> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Traditional: The video snippet discusses investing in stocks, shares, and the stock market, which are all traditional investments. The speaker also mentions the concept of capital gains, which is a traditional investment strategy. Although the video snippet does not explicitly recommend or educate about traditional investments, it provides information and insights that are relevant to traditional investments. Therefore, it falls under the Traditional label.",
        "role": "assistant"
      }
    }
  ],
  "created": 1677810191,
  "id": "chatcmpl-6ppMdZMjb0Pjkf2ooBsYVd6ZSBPno",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 77,
    "prompt_tokens": 596,
    "total_tokens": 673
  }
}